## Calculate the probabilities

In [5]:
def calculate_unigram_probs(tokens):
    train_probs = {}

    for i in range(0, len(train_tokens)):
        if (train_tokens[i] in list(train_probs.keys())):
            train_probs[train_tokens[i]] += 1
        else:
            train_probs[train_tokens[i]] = 1
    
    total = sum(list(train_probs.values()))
    for i in list(train_probs.keys()):
        train_probs[i] = train_probs[i]/total
    
    out = {
        'keys': np.array(list(train_probs.keys())),
        'values': np.array(list(train_probs.values()))
    }
    return out

In [144]:
def calculate_bigram_probs(tokens):
    bigram = {}
    for i in range(0, len(tokens)-1):
        if tokens[i] not in list(bigram.keys()):
            bigram[tokens[i]] = {}
        
        if tokens[i+1] not in list(bigram[tokens[i]].keys()):
            bigram[tokens[i]][tokens[i+1]] = 1
        else:
            bigram[tokens[i]][tokens[i+1]] += 1
        #print(i+1, tokens[i+1])
    
    for k in list(bigram.keys()):
        temp = np.array(list(bigram[k].values()))
        for l in list(bigram[k].keys()):
            bigram[k][l] = bigram[k][l] / temp.sum()
    
    return bigram

In [85]:
def calculate_trigram_probs(tokens):
    trigram = {}
    for i in range(0, len(tokens)-2):
        if tokens[i] not in list(trigram.keys()):
            trigram[tokens[i]] = {}
        
        if tokens[i+1] not in list(trigram[tokens[i]].keys()):
            trigram[tokens[i]][tokens[i+1]] = {}
        
        if tokens[i+2] not in list(trigram[tokens[i]][tokens[i+1]].keys()):
            trigram[tokens[i]][tokens[i+1]][tokens[i+2]] = 1
        else:
            trigram[tokens[i]][tokens[i+1]][tokens[i+2]] += 1
        #print(i+1, tokens[i+1])
    
    for k in list(trigram.keys()):
        for l in list(trigram[k].keys()):
            temp = np.array(list(trigram[k][l].values()))
            for m in list(trigram[k][l].keys()):
                trigram[k][l][m] = trigram[k][l][m] / temp.sum()
    
    return trigram

## Suggest words

In [8]:
def suggest_words_unigram(prob_values, prob_keys, word_, n = 5):
    
    subchar_indx = np.char.find(prob_keys, word_)
    count_ = 0
    for i in range(0, subchar_indx.shape[0]):
        if (subchar_indx[i] == 0):
            count_ += 1
    
    sub_idx_keys = np.zeros(count_, '<U23')
    sub_idx_values = np.zeros(count_, 'float64')
    
    count_ = 0
    for i in range(0, subchar_indx.shape[0]):
        if (subchar_indx[i] == 0):
            sub_idx_keys[count_] = prob_keys[i]
            sub_idx_values[count_] = prob_values[i]
            count_ += 1
    
    temp_probs = np.sort(sub_idx_values)[::-1]
    prob_list = list(sub_idx_values)
    
    if(n < len(temp_probs)):
        times = n
    else:
        times = len(temp_probs)
    
    out_list = [0]*times
    out_indxs = [0]*times
    
    for i in range(0, times):
        out_list[i] = sub_idx_keys[prob_list.index(temp_probs[i])]
        out_indxs[i] = prob_list.index(temp_probs[i])
    
    return out_list

In [132]:
def suggest_words_bigram(bigram_probs, word_, n = 5):
    
    words = word_.split(' ')
    words = words[(len(words)-2):(len(words))]
    
    if(len(words) == 0):
        words = [0]*2
        words[0] = "<s>"
        words[1] = ""
    elif (len(words) == 1):
        words.append("<s>")
        words.reverse()
        #print(words)
    #print(words)
    
    g_split = bigram_probs[words[0]]
    
    out_list = suggest_words_unigram(np.array(list(g_split.values())), np.array(list(g_split.keys())), words[1], n)
    
    return out_list

In [89]:
def suggest_words_trigram(trigram_probs, word_, n = 5):
    
    words = word_.split(' ')
    
    if (len(words) <= 2):
        start = 0
        end = len(words)
    else:
        start = len(words)-3
        end = len(words)
        
    words = words[start:end]
    #print(words)
    
    if (len(words) == 0):
        words = [0]*3
        words[0] = "<s>"
        words[1] = "<s>"
        words[2] = ""
    elif (len(words) == 1):
        words.append("<s>")
        words.append("<s>")
        words.reverse()
    elif (len(words) == 2):
        new = [0]*3
        new[0] = "<s>"
        new[1] = words[0]
        new[2] = words[1]
        words = new
    
    #print(words)
    g_split = trigram_probs[words[0]][words[1]]
    
    out_list = suggest_words_unigram(np.array(list(g_split.values())), np.array(list(g_split.keys())), words[2], n)
    
    return out_list